## Homework

In [1]:
import math
from ROOT import *
import numpy as np

# For inline image display
from IPython.display import Image

Welcome to JupyROOT 6.24/04


In [2]:
# Set ROOT graphics options

gROOT.SetStyle("Plain")

# Default canvas  size

gStyle.SetCanvasDefW(600)
gStyle.SetCanvasDefH(400)
 
# set the margins
  
gStyle.SetPadBottomMargin(0.18)
gStyle.SetPadTopMargin(0.08)
gStyle.SetPadRightMargin(0.08)
gStyle.SetPadLeftMargin(0.17)

# set the number of divisions to show

gStyle.SetNdivisions(510, "xy")

# show grid

gStyle.SetPadGridX(1)
gStyle.SetPadGridY(1)

# border ploted only around statistics

gStyle.SetPadBorderMode(0)
gStyle.SetFrameBorderMode(0)
gStyle.SetLegendBorderSize(0)
gStyle.SetTitleBorderSize(0)
gStyle.SetStatBorderSize(1)

# Text sizes

gStyle.SetTextSize(0.05)
gStyle.SetTitleSize(0.06,"xyz")
gStyle.SetLabelSize(0.05,"xyz")

gStyle.SetLabelOffset(0.015,"xyz")
gStyle.SetTitleOffset(1.2,"yz")
gStyle.SetTitleOffset(1.0,"x")

# Font settings

font = 42
gStyle.SetTitleFont(font)
gStyle.SetTitleFontSize(0.05)
gStyle.SetStatFont(font)
gStyle.SetStatFontSize(0.06)
gStyle.SetTextFont(font)
gStyle.SetLabelFont(font,"xyz")
gStyle.SetTitleFont(font,"xyz")
gStyle.SetLegendFont(font)

# Line settings

gStyle.SetHistLineWidth(2)
gStyle.SetFrameLineWidth(2)
gStyle.SetFuncWidth(2)
gStyle.SetHistLineColor(kBlue)
gStyle.SetFuncColor(kRed)

# Log scale
gStyle.SetOptLogy(0)    # 0 no log y scale, 1 log y scale

# Histogram title
gStyle.SetOptTitle(1)  

# Histogram statistics
gStyle.SetOptStat(1100)

# Fit results
gStyle.SetOptFit(0) 


In [3]:
# Class for single shower particle

class shParticle:
    'Particle of the EM shower (in 1D)'
    E = 0.0     # initial energy
    q = 0       # charge
    xmin = 0.0
    xmax = 0.0
    
    def __init__(self,E,q,xmin):
        self.E = E
        self.q = q
        self.xmin = xmin
        self.xmax = xmin
        
    def __str__(self):
        if self.xmin == self.xmax :
            return 'E = %g   q = % g    created at X = %g [X0]' % (self.E, self.q, self.xmin)
        else :
            return 'E = %g   q = % g    propagated from X = %g to %g  [X0]' % (self.E, self.q, self.xmin, self.xmax)
        
    def setlen(self,shlen):     # define endpoint of the particle; length after it decays
        self.xmax = self.xmin + shlen
 
    def getlen(self):       # define start of the particle
        return self.xmax - self.xmin
    
    def getsamp(self,dX):   # tells how long the track of the particle is in quantized manner
        i1 = int(self.xmin/dX)
        i2 = int(self.xmax/dX)
        return i2-i1


In [4]:
# EM shower class

class emShower:
    'Model of EM shower development'

    def __init__(self,E0,q0,x0):
        # initialize with input particle
        self.shower = []    # collection of particles
        self.shower.append(shParticle(E0,q0,x0))
        # only propagated particles are counted
        # so set counters to zero at that point
        self.npar = 0  
        self.nch = 0    # number of charged particles
        self.chlen = 0.0
        self.eloss = 0.0

    def __str__(self):
        return 'Shower with %d particles, %d charged, total track lenght %g X0, energy loss %g GeV' % (self.npar, self.nch, self.chlen, self.eloss)
    
    def develope(self,Ec,dEdx=0.0):
        # consider increasing number of particles in the shower
        for par in self.shower :
            # conversion/radiation length [X0]
            if par.q == 0 :     # check if it is charged
                intlen = 9.0/7.0
            else :  
                intlen = 1.0    # electron positron
               
            # generate conversion/radiation point
            
            shlen = np.random.exponential(intlen)
            
            # check energy loss of particle
            
            elen = 0
            
            if par.q != 0 and dEdx > 0 :
                elen = shlen * dEdx
                if elen > par.E :
                    shlen = par.E/dEdx
                    elen = par.E
            
            # propagate particle to this point
            
            par.setlen(shlen)
            self.npar+=1
            if par.q != 0 :
                self.nch+=1
                self.chlen+=shlen
                self.eloss+=elen

            # Energy at the final point
            
            Eleft = par.E - elen
            
            # Final point - origin for new particles
            
            xnew = par.xmax 
           
            # If above critical energy:
            #   convert gamma to two photons or radiate photon
            if Eleft > Ec :
                E1 = Eleft * np.random.random(1)    #  Very siplified energy splitting
                E2 = Eleft - E1
                if par.q == 0 :     # if it is photons
                    self.shower.append(shParticle(E1,+1,xnew))
                    self.shower.append(shParticle(E2,-1,xnew))
                else:       # electron positron
                    self.shower.append(shParticle(E1,par.q,xnew))
                    self.shower.append(shParticle(E2,0,xnew))
                    
    def sample(self,Xsamp):     # calculate how many samples are from the shower
        nsamp = 0
        for par in self.shower:
            if par.q != 0 :
                nsamp += par.getsamp(Xsamp)
        return nsamp

    def profile(self,Nbin,Xmax):        # shape of the shower as the histogram
        hprof = TH1D("hprof","Shower profile",Nbin, 0, Xmax)
        hprof.GetXaxis().SetTitle("x [X0]")
        hprof.GetYaxis().SetTitle("Particles")
        dX=Xmax/Nbin
        for par in self.shower:
            if par.q != 0 :
                nx = par.getsamp(dX)
                for ix in range(nx):
                    hprof.Fill(par.xmin+ix*dX)
        return hprof
    
    def dump(self,Nprt):
        for iprt in range(Nprt):
            print(self.shower[iprt])
            

## Our shower simulation

In [5]:
#  Incident particle

E0 = 1.
q0 = -1

# Initialise shower with incident particle

myshower = emShower(E0,q0,0.)

print(myshower.shower[0])

E = 1   q = -1    created at X = 0 [X0]


#### Fitting function

In [6]:
def fitfunc(x0, y0, sigma, x):
    return y0 * np.exp(- (x0 / sigma) ** 2 * ((x - x0) / x0) - np.log(x / x0))

##### Test fitfunc

In [8]:
a = fitfunc(1, 2, 3, 4)
a

0.3582656552868947